# Veri Analizleri

In [2]:
import pandas as pd
import numpy as np
from scipy import stats

In [3]:
from plotly import express
from plotly import graph_objects
from plotly import io

io.templates.default = "plotly_dark"

In [4]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.tsa.stattools import adfuller, acf, breakvar_heteroskedasticity_test

In [5]:
from statsmodels.graphics import tsaplots
import matplotlib.pyplot as plt

In [6]:
import plotly.graph_objects as go
import statsmodels.api as sm 

In [7]:
country_name = ["France", "Greece",	"Italy", "Portugal", "Spain", "Türkiye"]

In [8]:
df_2 = pd.read_excel("OECD VERİLERİ ELİF BAĞCI TEZ V0_python.xlsx", sheet_name=3, header=1 )
df_2.head()

,Unnamed: 0,France,Greece,Italy,Portugal,Spain,Türkiye,t,interruption,interrupted t
0,2000,2687.530,1418.418,2030.535,1642.936,1524.076,432.043,1,0,0
1,2001,2875.294,1700.929,2151.727,1709.799,1635.587,472.841,2,0,0
2,2002,3152.016,1951.129,2292.714,1839.118,1803.500,519.978,3,0,0
3,2003,3056.265,2048.654,2289.348,1921.970,2015.047,509.726,4,0,0
4,2004,3170.947,2100.803,2451.044,2085.322,2123.064,558.587,5,0,0


In [9]:
def create_corr_plot(series, plot_pacf=False):
    corr_array = pacf(series.dropna(), alpha=0.05) if plot_pacf else acf(series.dropna(), alpha=0.05)
    lower_y = corr_array[1][:,0] - corr_array[0]
    upper_y = corr_array[1][:,1] - corr_array[0]

    fig = go.Figure()
    [fig.add_scatter(x=(x,x), y=(0,corr_array[0][x]), mode='lines',line_color='#3f3f3f') 
     for x in range(len(corr_array[0]))]
    fig.add_scatter(x=np.arange(len(corr_array[0])), y=corr_array[0], mode='markers', marker_color='#1f77b4',
                   marker_size=12)
    fig.add_scatter(x=np.arange(len(corr_array[0])), y=upper_y, mode='lines', line_color='rgba(255,255,255,0)')
    fig.add_scatter(x=np.arange(len(corr_array[0])), y=lower_y, mode='lines',fillcolor='rgba(32, 146, 230,0.3)',
            fill='tonexty', line_color='rgba(255,255,255,0)')
    fig.update_traces(showlegend=False)
    fig.update_xaxes(range=[-1,13])
    fig.update_yaxes(zerolinecolor='#000000')
    
    title='Partial Autocorrelation (PACF)' if plot_pacf else 'Autocorrelation (ACF)'
    fig.update_layout(title=title)
    fig.show()

In [10]:
for country in country_name:
    print("###########################")
    print(f"{country}'nin Zaman Serisi Analizi")
    print("###########################")

    trendlin = express.scatter(
        df_2,
        x="t",
        y=f"{country}",
        title=f"{country}'s Current expenditure on health, per capita, US$ purchasing power parities (current prices, current PPPs)",
        trendline = "ols"
    )

    trendlin.show()
    linear_model = ols(f"{country} ~ t", data=df_2).fit()
    print(linear_model.summary())
    print(" ", end="\n")
    print("###########################")
    print("Verinin Durağanlık Kontrolü", end="\n\n")
    print("H0: Seri durağan değildir.")
    print("H1: Seri durağandır.", end="\n\n")
    adfullerx = adfuller(df_2[[f"{country}"]])
    print(f'Test Stat = {round(adfullerx[0], 4)}, p-value = {round(adfullerx[1], 4)}')

    if adfullerx[1]<0.05:
        print("p-value 0.05'ten küçüktür. H0 hipotezi reddedilmiştir. Yani seri durağandır.")
    elif adfullerx[1]>0.05:
        print("p-value 0.05'ten büyüktür. H0 hipotezi reddedilememiştir. Yani seri durağan değildir.", end="\n\n")
    print("###########################")
    print("Hataların Normal Dağılması", end="\n\n")
    residual_values = linear_model.resid

    print("H0: Normal dağılım varsayımı sağlanmaktadır.")
    print("H1: Normal dağılım varsayımı sağlanmamaktadır.", end="\n\n")

    test_stat, pvalue = stats.shapiro(residual_values)
    print(f'Test Stat = {round(test_stat, 4)}, p-value = {round(pvalue, 4)}')
    if pvalue<0.05:
        print("p-value 0.05'ten küçüktür. H0 hipotezi reddedilmiştir. Hatalar normal dağılmamıştır.")
    elif pvalue>0.05:
        print("p-value 0.05'ten büyüktür. H0 hipotezi reddedilememiştir. Yani hatalar normal dağılmıştır.", end="\n\n")
    hist = express.histogram(residual_values, title="Hataların Dağılımı")
    hist.show()

    print("###########################")
    print("Hataların Rassallığı", end="\n\n")

    print("     Hataların Durağanlığı", end="\n\n")

    adfullery = adfuller(residual_values)
    print(f'Test Stat = {round(adfullery[0], 4)}, p-value = {round(adfullery[1], 4)}')

    if adfullery[1]<0.05:
        print("p-value 0.05'ten küçüktür. H0 hipotezi reddedilmiştir. Hatalar durağandır.")
    elif adfullery[1]>0.05:
        print("p-value 0.05'ten büyüktür. H0 hipotezi reddedilememiştir. Yani hatalar durağan değildir.", end="\n\n")

    resid_scatter = express.scatter(
        df_2,
        x="t",
        y=residual_values,
        title="Hataların Rassallığı"
    )

    resid_scatter.show()


    print("     Hataların Otokorelasyonu", end="\n\n")

    print("Ljung-Box q istatistiğinin Hipotezleri")
    print("H0: Veriler bağımsız (random) olarak dağılmıştır.")
    print("H1: Veriler bağımsız (random) olarak dağılmamıştır.", end="\n\n")

    acf_df = pd.DataFrame(pd.DataFrame(acf(residual_values, qstat=True)).T)
    acf_df.columns = ["ACF", "Ljung-Box q statistic", "p-value"]
    display(acf_df[1:6])

    create_corr_plot(residual_values)


###########################
France'nin Zaman Serisi Analizi
###########################


                            OLS Regression Results                            
Dep. Variable:                 France   R-squared:                       0.993
Model:                            OLS   Adj. R-squared:                  0.993
Method:                 Least Squares   F-statistic:                     2756.
Date:                Tue, 31 Jan 2023   Prob (F-statistic):           4.96e-22
Time:                        19:10:47   Log-Likelihood:                -118.43
No. Observations:                  21   AIC:                             240.9
Df Residuals:                      19   BIC:                             242.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   2552.0013     32.380     78.813      0.0

###########################
Hataların Rassallığı

     Hataların Durağanlığı

Test Stat = -1.9044, p-value = 0.33
p-value 0.05'ten büyüktür. H0 hipotezi reddedilememiştir. Yani hatalar durağan değildir.



     Hataların Otokorelasyonu

Ljung-Box q istatistiğinin Hipotezleri
H0: Veriler bağımsız (random) olarak dağılmıştır.
H1: Veriler bağımsız (random) olarak dağılmamıştır.



,ACF,Ljung-Box q statistic,p-value
1,0.157602,0.763936,0.682517
2,-0.080342,0.917669,0.821162
3,-0.075692,0.917670,0.922012
4,0.000082,2.881843,0.718196
5,-0.255080,3.989235,0.678133


...

###########################
Türkiye'nin Zaman Serisi Analizi
###########################


                            OLS Regression Results                            
Dep. Variable:                Türkiye   R-squared:                       0.992
Model:                            OLS   Adj. R-squared:                  0.991
Method:                 Least Squares   F-statistic:                     2272.
Date:                Tue, 31 Jan 2023   Prob (F-statistic):           3.06e-21
Time:                        19:10:48   Log-Likelihood:                -96.670
No. Observations:                  21   AIC:                             197.3
Df Residuals:                      19   BIC:                             199.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    366.9621     11.490     31.938      0.0

###########################
Hataların Rassallığı

     Hataların Durağanlığı

Test Stat = -2.0055, p-value = 0.2842
p-value 0.05'ten büyüktür. H0 hipotezi reddedilememiştir. Yani hatalar durağan değildir.



     Hataların Otokorelasyonu

Ljung-Box q istatistiğinin Hipotezleri
H0: Veriler bağımsız (random) olarak dağılmıştır.
H1: Veriler bağımsız (random) olarak dağılmamıştır.



,ACF,Ljung-Box q statistic,p-value
1,0.429263,4.578964,0.101319
2,0.071213,8.925916,0.030292
3,-0.402490,17.360914,0.001644
4,-0.544871,24.509814,0.000173
5,-0.486638,24.730434,0.000383


# Interrupted Time Series for Türkiye

    Yukarıda referans ülkelerin ve Türkiye'nin zaman serisi regresyon analizi sonuçlarına bakılırken, bu kısımdan itibaren Türkiye için interrupted time series analysis yapılmıştır.

In [11]:
X = df_2[["t", "interruption", "interrupted t"]]
y = df_2["Türkiye"]

X = sm.add_constant(X) 

analys= sm.OLS(y, X).fit()

trendlin = express.scatter(
        df_2,
        x="t",
        y= "Türkiye",
        title="Current expenditure on health, per capita, US$ purchasing power parities (current prices, current PPPs)",
    )

trendlin.add_trace(
go.Scatter(
    x = [1,5],
    y = [analys.predict()[0], analys.predict()[4]],
)
 )   

trendlin.add_trace(
go.Scatter(
    x = [5,6],
    y = [analys.predict()[4], analys.predict()[5]]
)

 )

trendlin.add_trace(
go.Scatter(
    x = [6,21],
    y = [analys.predict()[5], analys.predict()[20]]
)

 )  

trendlin.show() 



print(analys.summary())

print("###########################")
print("Hataların Normal Dağılması", end="\n\n")
residual_values = analys.resid
print("H0: Normal dağılım varsayımı sağlanmaktadır.")
print("H1: Normal dağılım varsayımı sağlanmamaktadır.", end="\n\n")
test_stat, pvalue = stats.shapiro(residual_values)
print(f'Test Stat = {round(test_stat, 4)}, p-value = {round(pvalue, 4)}')
if pvalue<0.05:
    print("p-value 0.05'ten küçüktür. H0 hipotezi reddedilmiştir. Hatalar normal dağılmamıştır.")
elif pvalue>0.05:
     print("p-value 0.05'ten büyüktür. H0 hipotezi reddedilememiştir. Yani hatalar normal dağılmıştır.", end="\n\n")
hist = express.histogram(residual_values, title="Hataların Dağılımı")
hist.show()
print("###########################")
print("Hataların Rassallığı", end="\n\n")
print("     Hataların Durağanlığı", end="\n\n")
adfullery = adfuller(residual_values)
print(f'Test Stat = {round(adfullery[0], 4)}, p-value = {round(adfullery[1], 4)}')
if adfullery[1]<0.05:
    print("p-value 0.05'ten küçüktür. H0 hipotezi reddedilmiştir. Hatalar durağandır.")
elif adfullery[1]>0.05:
    print("p-value 0.05'ten büyüktür. H0 hipotezi reddedilememiştir. Yani hatalar durağan değildir.", end="\n\n")
resid_scatter = express.scatter(
    df_2,
    x="t",
    y=residual_values,
    title="Hataların Rassallığı"
)
resid_scatter.show()
print("     Hataların Otokorelasyonu", end="\n\n")
print("Ljung-Box q istatistiğinin Hipotezleri")
print("H0: Veriler bağımsız (random) olarak dağılmıştır.")
print("H1: Veriler bağımsız (random) olarak dağılmamıştır.", end="\n\n")
acf_df = pd.DataFrame(pd.DataFrame(acf(residual_values, qstat=True)).T)
acf_df.columns = ["ACF", "Ljung-Box q statistic", "p-value"]
display(acf_df[1:6])
create_corr_plot(residual_values)

                            OLS Regression Results                            
Dep. Variable:                Türkiye   R-squared:                       0.993
Model:                            OLS   Adj. R-squared:                  0.992
Method:                 Least Squares   F-statistic:                     831.2
Date:                Tue, 31 Jan 2023   Prob (F-statistic):           1.24e-18
Time:                        19:10:48   Log-Likelihood:                -94.542
No. Observations:                  21   AIC:                             197.1
Df Residuals:                      17   BIC:                             201.3
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const           411.6431     25.441     16.180

###########################
Hataların Rassallığı

     Hataların Durağanlığı

Test Stat = -2.2659, p-value = 0.1832
p-value 0.05'ten büyüktür. H0 hipotezi reddedilememiştir. Yani hatalar durağan değildir.



     Hataların Otokorelasyonu

Ljung-Box q istatistiğinin Hipotezleri
H0: Veriler bağımsız (random) olarak dağılmıştır.
H1: Veriler bağımsız (random) olarak dağılmamıştır.



,ACF,Ljung-Box q statistic,p-value
1,0.296047,2.388933,0.302865
2,0.103503,4.591616,0.204262
3,-0.286509,8.922703,0.063061
4,-0.390436,13.432324,0.019647
5,-0.386506,14.188557,0.027599
